In [217]:
import requests
from bs4 import BeautifulSoup
import json

# URL de la página
url = "https://en.wikipedia.org/wiki/List_of_Puerto_Rico_landmarks"

# Realizar la solicitud
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Inicializar las estructuras de datos
data = {"sections": []}

h2_and_lists = []
capture = False

content_div = soup.select_one("#mw-content-text > div.mw-content-ltr.mw-parser-output")
start_id = "Adjuntas"

print('Processing landmarks',end=' ')
for element in content_div.find_all(['h2', 'ul']):
    # print(element)
    print('.', end='')

    if element == 'See_also' and capture:
        break
    

    if element.name in ['h2']:
        text = element.get_text(separator=' ', strip=True)

    # if capture and element.name in ['ul']:
    if element.name in ['ul']:
        list_items = [li.get_text(separator=' ', strip=True) for li in element.find_all('li')]
        list_items = list(filter(bool, list_items))
        if list_items:
            # formatted_list = "\n  - " + "\n  - ".join(list_items)
            for item in list_items:
                data["sections"].append({
                    "place": item,
                    "city": text
                })

print("\nData transformation successfully.")

Processing landmarks .....................................................................................................................................................................................................
Data successfully created.


In [221]:
# Test to find Landmark City

place_name = 'Casa Pueblo'

city_object = next((section for section in data["sections"] if section["place"] == place_name), None)

print(city_object)
print(len(data['sections']))    

{'place': 'Casa Pueblo', 'city': 'Adjuntas'}
945


In [331]:
import requests
from bs4 import BeautifulSoup
import json
import re

# URL base y página inicial
base_url = "https://en.wikipedia.org"
start_url = "https://en.wikipedia.org/wiki/List_of_Puerto_Rico_landmarks"

# Realizar la solicitud a la página principal
response = requests.get(start_url)
soup = BeautifulSoup(response.content, 'html.parser')

def clean_content(text):
    # Replace accented characters and special cases
    replacements = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
        'ñ': 'n', 'Ñ': 'N', 'ü': 'u', 'Ü': 'U'
    }
    
    # Replace each character in the dictionary
    for accented_char, replacement_char in replacements.items():
        text = text.replace(accented_char, replacement_char)

    # Remove references like [1], [ 99 ], etc.
    text = re.sub(r'\[\s*[a-z0-9]*\s*\]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)

    # Return cleaned text
    return text.strip()

def get_general_paragraphs(soup):
    content_div = soup.select_one("#mw-content-text > div.mw-parser-output")
    if content_div:
        paragraphs = content_div.find_all("p")
        return [clean_content(p.get_text(separator=' ', strip=True)) for p in paragraphs if p.get_text(strip=True)]
    return []

# Inicializar estructura para almacenar los datos
landmarks = []

print('Processing ', end='')
# Buscar todos los enlaces de los lugares destacados
for link in soup.select(".mw-parser-output ul li a"):    
    place_name = link.get_text(separator=' ', strip=True)
    # print(f"{place_name}", end=', ')
    print(f".", end='')
    
    relative_url = link['href']
    full_url = base_url + relative_url

    # Realizar la solicitud a cada página de lugar
    place_response = requests.get(full_url)
    place_soup = BeautifulSoup(place_response.content, 'html.parser')

    # Extraer descripción breve del primer párrafo
    # description_tag = place_soup.find_all('p')
    general_info = get_general_paragraphs(place_soup)

    if len(general_info) > 0:
        description = '  '.join(general_info[:2])
    
    # Extraer imagen preview (si existe)
    image_tag = place_soup.find('img')
    image_url = "https:" + image_tag['src'] if image_tag else "No image available"

    # print(place_name, end='  ')
    city_object = next((section for section in data["sections"] if place_name in section["place"]), None)

    # print(city_object)
    # Guardar en la estructura de datos
    landmarks.append({
        "content": description,        
        "metadata": {
            "type": "landmark",
            "city": city_object["city"],
            "name": city_object["place"],
            "categories" : [],
            "url": full_url,            
            "image_url": image_url
        }
    })

# print(landmarks[0])
print("\nData successfully created.")

Processing .......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
Data successfully created.


In [333]:
# Save the data into a JSON File
with open('./datasets/puerto_rico_landmarks.json', 'w') as file:
    json.dump(landmarks, file, indent=4)

print('Data saved to puerto_rico_landmarks.json')

Data saved to puerto_rico_landmarks.json
